## Лабораторная работа по модулю “Построение моделей”

<pre>
ФИО: Никифоров Владимир
</pre>

В данной работе предлагается решить задачу мультиклассовой классификации для датасета флагов различных стран
(https://archive.ics.uci.edu/ml/datasets/Flags)
1. Загрузить данные из папки Data Folder
2. Провести описательный анализ предложенных данных
3. Предложить стратегию по работе с категориальными переменными
4. Провести углубленный анализ данных (корреляции переменных, визуализация взаимосвязей)
5. Следует ли увеличивать/уменьшать размерность признакового множества (какой подход выдает лучшие результаты)
6. Следует ли балансировать классы путем oversampling/undersampling? (улучшает ли подход результаты)
7. Сделать кросс-валидацию данных с использованием подхода K-fold (n_folds=3)
8. Решить задачу мультиклассовой классификации и предсказать религию страны (religion) по деталям ее флага
9. Рассчитать Feature Importances для переменных модели
10. Проверить качество классификации с использованием следующих метрик: Accuracy, F1-Score, Precision, Recall
11. Построить Confusion Matrix для ваших результатов
12. Загрузить ipython notebook с результатами работы на github репозиторий
*Пункты, показавшиеся вам сложными, либо которые вы не знаете можно пропускать, но с пометкой в ноутбуке, что
показалось сложным и непонятным в данном пункте для вас.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd


from scipy.stats import pearsonr
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score, accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

/home/truename/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


ModuleNotFoundError: No module named 'imblearn'

In [ ]:
!cat ../data/flag.names

In [ ]:
RANDOM_STATE = 7
class_names=['Catholic', 'Other Christian', 'Muslim', 'Buddhist', 'Hindu','Ethnic', 'Marxist', 'Others']

In [ ]:
raw_df = pd.read_csv('../data/flag.data',header=0,names=['name','landmass','zone','area','population','language','religion','bars','stripes','colours','red','green','blue','gold','white','black','orange','mainhue','circles','crosses','saltires','quarters','sunstars','crescent','triangle','icon','animate','text','topleft','botright'])
print(f'Shape of dataset: {raw_df.shape}')
raw_df.head()

#['name','landmass','zone','area','population','language','religion','bars','stripes','colours','red','green','blue','gold','white','black','orange','mainhue','circles','crosses','saltires','quarters','sunstars','crescent','triangle','icon','animate','text','topleft','botright']

In [ ]:
raw_df.info()

Как и сказали в описании, пропусков нет, и на том спасибо)

In [ ]:
raw_df.describe().T

In [ ]:
fig, ax = plt.subplots(figsize=(30,20))
sns.heatmap(raw_df.corr(),annot=True,cmap="RdYlGn");

In [ ]:
g=sns.pairplot(data=raw_df,plot_kws={'alpha':0.2});
g.fig.set_size_inches(30,30)

В датасете присутствует много значимых признаков (большой разброс). Т.к. необходимо решить задачу мультиклассовой классификации по признаку religion - посмотрим на все признаки в разрезе religion отдельно. Сразу стоит отметить о несбалансированности классов в призаке religion.

In [ ]:
g=sns.pairplot(data=raw_df,plot_kws={'alpha':0.2}, hue='religion');
g.fig.set_size_inches(30,30)

Кроме очевидных территории и языка, также можно заметить признаки наличия различных полос, перекрестий на флагах, которые имеют различные распределения для разных значений признака religion.

In [ ]:
# К примеру:
sns.jointplot(data=raw_df,y='religion',x='crosses',kind='reg',stat_func=pearsonr);

Попробуем обойтись без уменьшения размерности признакового множества, чтобы не потерять часть информации.

In [ ]:
# Будем использовать кросс-валидацию данных с K-fold
N_FOLDS=3

In [ ]:
raw_df.columns

In [ ]:
X, y = raw_df[['landmass', 'zone', 'area', 'population', 'language',
       'bars', 'stripes', 'colours', 'red', 'green', 'blue',
       'gold', 'white', 'black', 'orange', 'mainhue', 'circles', 'crosses',
       'saltires', 'quarters', 'sunstars', 'crescent', 'triangle', 'icon',
       'animate', 'text', 'topleft', 'botright']], raw_df['religion']

In [ ]:
# Категориальные признаки разобъем на детальные признаки, созданные по значениям:
X = pd.get_dummies(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= RANDOM_STATE)

# Т.к. классы признака religion несбаллансированы, то будем использовать oversampling для баллансировки классов.
sm = SMOTE(random_state=12, ratio = 1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)

In [ ]:
cross_val_score( DecisionTreeClassifier( max_depth = 15, random_state= RANDOM_STATE ), X, y, cv = N_FOLDS, scoring = 'accuracy' )

In [ ]:
params = [ {'max_depth': list( range(1, 20) )} ]
gs = GridSearchCV( DecisionTreeClassifier(random_state= RANDOM_STATE), param_grid = params, scoring = 'accuracy', return_train_score = False )
gs.fit( X, y )
print('cv_results_',gs.cv_results_)
print('best_params_',gs.best_params_)

In [ ]:
# посмотрим на качество классификации при наилучшей глубине gs.best_params_['max_depth']
tc = DecisionTreeClassifier( max_depth = gs.best_params_['max_depth'], random_state= RANDOM_STATE)
cross_val_score( tc, X, y, cv = N_FOLDS, scoring = 'accuracy' )
tc.fit(X,y)

In [ ]:
# Важность переменных:
features = X.columns
importances = tc.feature_importances_[0:20]

indices = np.argsort(importances)

plt.title('Feature Importances for DecisionTree')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest_params = {
    'n_estimators': [5, 10, 20, 40],
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 3, 5, 7, 9, None],
    'min_samples_leaf': [1, 2, 4, 8, 16]
}

In [ ]:
forest = RandomForestClassifier(random_state=RANDOM_STATE)
grid_forest = GridSearchCV(forest, forest_params, scoring='accuracy', cv=N_FOLDS, n_jobs=-1)
grid_forest.fit(X_train, y_train)

In [ ]:
print(grid_forest.best_params_)
print(grid_forest.best_score_)
print(grid_forest.best_estimator_)

Best_score_ = 0.6592592592592592 - показатель лучше, чем у одного дерева решений!

In [ ]:
model_rf = RandomForestClassifier(criterion='gini', max_depth= 9, min_samples_leaf= 1, n_estimators= 20, random_state=RANDOM_STATE)
model_rf.fit(X_train, y_train)

In [ ]:
# Важность переменных:
features = X.columns
importances = model_rf.feature_importances_[0:20]

indices = np.argsort(importances)

plt.title('Feature Importances for RandomForest')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
y_pred = model_rf.predict(X_test)

In [ ]:
print('Accuracy=',accuracy_score(y_test, y_pred))
print('f1_score=',f1_score(y_test, y_pred, average="macro"))
print('precision=',precision_score(y_test, y_pred, average="macro"))
print('recall=',recall_score(y_test, y_pred, average="macro"))  

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names))

## Confusion matrix

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
df = pd.DataFrame({'y_Actual': y_test.values, 'y_Predicted': y_pred}, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])

ax=sns.heatmap(confusion_matrix, annot=True);
ax.set_yticklabels(class_names, rotation=0);
ax.set_xticklabels(class_names, rotation=90);

## Посмотрим, что мы можем улучшить с помощью понижения размерности:

In [ ]:
print(f'Текущая размерность датасета: {X_train.shape}')

In [ ]:
# Возьмем для начала сжатие до 10 компонент
n=10
pca = PCA(n_components=n)
pc = pca.fit_transform(pd.get_dummies(raw_df.drop(['religion'], axis=1,inplace=False)))
df_pc = pd.DataFrame(data = pc, columns = ['pc'+str(i) for i in range(n)])
df_with_pc = pd.concat([df_pc, raw_df[['religion']]], axis = 1)

Теперь натравим на этот датасет случайный лес (как наилучший алгоритм на тесте):

In [ ]:
X, y = df_with_pc.drop(['religion'], axis=1,inplace=False), df_with_pc['religion'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25)

# Т.к. классы признака religion несбаллансированы, то будем использовать oversampling для баллансировки классов.
sm = SMOTE(random_state=12, ratio = 1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE).fit(X_train, y_train)
y_test_predict = clf.predict(X_test)
print(f'accuracy: {accuracy_score(y_test, y_test_predict)}')
print(f'f1: {f1_score(y_test, y_test_predict, average="macro")}')
print(f'precision: {precision_score(y_test, y_test_predict, average="macro")}')
print(f'recall: {recall_score(y_test, y_test_predict, average="macro")}')

Лучше не стало. Может 10 компонент - не самое оптимальное количество? Посмотрим на все возможные комбинации и результаты на тесте:

In [ ]:
l_result = []
for n in range(1,raw_df.shape[1]-1):
    pca = PCA(n_components=n, random_state=RANDOM_STATE)
    pc = pca.fit_transform(pd.get_dummies(raw_df.drop(['religion'], axis=1,inplace=False)))
    df_pc = pd.DataFrame(data = pc, columns = ['pc'+str(i) for i in range(n)])
    df_with_pc = pd.concat([df_pc, raw_df[['religion']]], axis = 1)
    
    X, y = df_with_pc.drop(['religion'], axis=1,inplace=False), df_with_pc['religion'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25)
    
    clf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1).fit(X_train, y_train)
    y_test_predict_proba = clf.predict_proba(X_test)[:, 1]
    y_test_predict = clf.predict(X_test)
    l_result.append((n,accuracy_score(y_test, y_test_predict),f1_score(y_test, y_test_predict, average="macro")))

In [ ]:
# отсортируем по наивысшей F-мере
l_result.sort(key=lambda x:x[2],reverse=True)
l_result

In [ ]:
print(f'Лучшее количество компонент: {l_result[0][0]}')

In [ ]:
# Проверим лучшее сжатие
n=l_result[0][0]
pca = PCA(n_components=n)
pc = pca.fit_transform(pd.get_dummies(raw_df.drop(['religion'], axis=1,inplace=False)))
df_pc = pd.DataFrame(data = pc, columns = ['pc'+str(i) for i in range(n)])
df_with_pc = pd.concat([df_pc, raw_df[['religion']]], axis = 1)

Теперь натравим на этот датасет случайный лес (как наилучший алгоритм на тесте):

In [ ]:
X, y = df_with_pc.drop(['religion'], axis=1,inplace=False), df_with_pc['religion'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25)

# Т.к. классы признака religion несбаллансированы, то будем использовать oversampling для баллансировки классов.
sm = SMOTE(random_state=12, ratio = 1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE).fit(X_train, y_train)
y_test_predict = clf.predict(X_test)
print(f'accuracy: {accuracy_score(y_test, y_test_predict)}')
print(f'f1: {f1_score(y_test, y_test_predict, average="macro")}')
print(f'precision: {precision_score(y_test, y_test_predict, average="macro")}')
print(f'recall: {recall_score(y_test, y_test_predict, average="macro")}')